# 1. Data Loading and Exploration

This notebook demonstrates how to load and explore the multi-modal data from the Dareplane project. We will cover:

- Loading XDF files for a single session.
- Inspecting the `mne.Raw` object containing LFP, ECoG, and EOG data.
- Visualizing the raw neural data and event markers.
- Extracting and visualizing the stylus tracing coordinates from the behavioral data stream.

## 1.1 Setup

First, let's import the necessary libraries and add the `analysis_scripts` directory to our Python path to access the custom loading functions.

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px
import mne
import pyxdf

# Add the analysis_scripts directory to the Python path
sys.path.append(str(Path.cwd().parent.parent / 'analysis_scripts'))

from load_xdf import get_xdf_files, create_raws_from_mat_and_xdf

## 1.2 Load Neural Data

Now, we'll load the data for a specific session. The `get_xdf_files` function helps us find the relevant XDF files, and `create_raws_from_mat_and_xdf` loads the data into a list of `mne.Raw` objects, one for each block in the session.

In [ ]:
# Specify the session to load (e.g., 'day3', 'day4', 'day5')
# Note: The original data seems to be in folders like data_p001, data_p002, etc.
# For this example, we will assume a session identifier that maps to these folders.
# Let's use 'S003' as an example session, which we'll map to 'data_p003'
session_id = 'S003' # Example session
session_folder = f'data_p00{session_id[-1]}'

xdf_files = get_xdf_files(session_folder)

# The 'day' parameter in create_raws_from_mat_and_xdf seems to be used for handling
# session-specific alignment issues. We'll use a placeholder value for now.
raws = create_raws_from_mat_and_xdf(xdf_files, day='day3') # Using 'day3' as per function's default logic

# For demonstration, let's concatenate all blocks into a single Raw object
raw = mne.concatenate_raws(raws)

## 1.3 Explore the `mne.Raw` Object

The loaded data is now in an `mne.Raw` object. MNE-Python provides a rich interface for exploring this data.

In [ ]:
print(raw.info)

In [ ]:
print(f"Channel names: {raw.ch_names}")
print(f"Number of channels: {len(raw.ch_names)}")
print(f"Sampling frequency: {raw.info['sfreq']} Hz")

## 1.4 Visualize Raw Data and Events

Let's plot the raw time series data. We can also visualize the event markers, which are stored as `Annotations` in the `mne.Raw` object.

In [ ]:
# Plot the raw data (it's interactive!)
# We'll select a subset of channels and a short duration for clarity
picks = mne.pick_channels(raw.ch_names, include=['LFP_1', 'LFP_2', 'ECOG_1', 'ECOG_2'])
raw.plot(start=0, duration=10, picks=picks, n_channels=4, scalings='auto')

In [ ]:
# The event markers are stored in raw.annotations
print(raw.annotations)

## 1.5 Extract and Visualize Stylus Tracing Data

The stylus (tracing) coordinates are stored in a separate stream within the XDF file. We need to load the XDF file again with `pyxdf` to access all streams and find the one containing the stylus data. Based on the task (CopyDraw), we can look for a stream with a name related to drawing or mouse coordinates.

In [ ]:
# Let's use the first XDF file from the session for this example
xdf_file_path = xdf_files[0]
streams, header = pyxdf.load_xdf(xdf_file_path)

stylus_stream = None
for stream in streams:
    # Heuristic to find the stylus stream: it might be named 'Mouse' or similar,
    # and will likely have 2 or 3 channels (X, Y, maybe pressure).
    stream_name = stream['info']['name'][0]
    if 'Mouse' in stream_name or 'Stylus' in stream_name or 'Coordinates' in stream_name:
        stylus_stream = stream
        break

if stylus_stream:
    print(f"Found stylus stream: {stylus_stream['info']['name'][0]}")
    stylus_data = stylus_stream['time_series']
    stylus_timestamps = stylus_stream['time_stamps']

    # Create a pandas DataFrame for easier handling
    df_stylus = pd.DataFrame(stylus_data, columns=['x', 'y']) # Assuming 2D coordinates
    df_stylus['time'] = stylus_timestamps

    # Plot the stylus trace
    fig = px.line(df_stylus, x='x', y='y', title='Stylus Trace')
    fig.update_yaxes(autorange="reversed") # Often, screen Y coordinates are inverted
    fig.show()
else:
    print("Could not automatically find a stylus/mouse stream in the XDF file.")
    print("Available streams:")
    for stream in streams:
        print(f"- {stream['info']['name'][0]}")